# The TypeformSync in debug mode

This notebook runs the TypeformSync module but lets you debug it.

Notebook and module written by Avi Alkalay &lt;avi at unix.sh&gt;

São Paulo, July 2019

In [ ]:
from configobj import ConfigObj    # dnf install python3-configobj
import logging
import pandas as pd

config='syncFromTypeform.conf'

## Import the module

In [ ]:
from Typeform import TypeformSync

## Initialize logger and sync module

In [ ]:
context=ConfigObj(config)

# Switch between INFO/DEBUG while running in production/developping:
logging.getLogger().setLevel(logging.DEBUG)
logging.getLogger('Typeform').setLevel(logging.DEBUG)
logging.getLogger('urllib3.connectionpool').setLevel(logging.WARNING)


tf = TypeformSync(context['typeform_token'],context['database'])

## Sync

In [ ]:
%time tf.sync()

## Some debugging from here...

In [ ]:
logging.getLogger('Typeform').setLevel(logging.DEBUG)
# print(logging.getLogger('TypeForm').__dict__)

str(logging.getLogger('TypeForm').__dict__)

tf.logger.debug('abc')
logging.debug('123')

In [ ]:
logging.getLogger('Typeform').__dict__
tf.logger.debug(123)

In [ ]:
answers_db=pd.read_sql_table(con=tf.db,table_name='answers')

In [ ]:
answers_db.set_index('id', inplace=True)

In [ ]:
df = pd.merge(tf.answers, answers_db, how='outer', indicator=True)
rows_in_tf_not_in_db = df[df['_merge']=='left_only']
rows_in_db_not_in_tf = df[df['_merge']=='right_only']
tf.answers.sort_values(by='id').head(50)

In [ ]:
answers_db.sort_values(by='id').head(50)

In [ ]:
answers_db.shape

In [ ]:
# On a first sync, this should get you an empty DataFrame, meaning that DataFrame built from Typeform API and DB table are identical:
rows_in_tf_not_in_db

In [ ]:
# On a first sync, this should get you an empty DataFrame, meaning that DataFrame built from Typeform API and DB table are identical:
rows_in_db_not_in_tf

In [ ]:
tf.answers.data_type_hint.value_counts()

In [ ]:
tf.responses.sort_values('submitted')

In [ ]:
tf.answers['response']=pd.Categorical(tf.answers['response'],tf.responses.sort_values(by='landed').index)
tf.answers.sort_values(by='response')

In [ ]:
tf.formItems.type.value_counts()